##  Implement Decision Tree

In [1]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd

In [33]:
#Load dataset
iris = load_iris()
#Features
X = iris.data.copy()
#Classes
y = iris.target.reshape(150,1)
#Split dataset into train/test
X_train,X_test, y_train, y_test = train_test_split(X,y, test_size = 0.33)
#Horizontally stacking the train_dataset together
train_set = np.hstack((X_train, y_train))

print("features:", X[0:10])
print("labels:", y[0:10])
print(train_set[0:10])

features: [[ 5.1  3.5  1.4  0.2]
 [ 4.9  3.   1.4  0.2]
 [ 4.7  3.2  1.3  0.2]
 [ 4.6  3.1  1.5  0.2]
 [ 5.   3.6  1.4  0.2]
 [ 5.4  3.9  1.7  0.4]
 [ 4.6  3.4  1.4  0.3]
 [ 5.   3.4  1.5  0.2]
 [ 4.4  2.9  1.4  0.2]
 [ 4.9  3.1  1.5  0.1]]
labels: [[0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]]
[[ 4.7  3.2  1.6  0.2  0. ]
 [ 6.3  2.5  4.9  1.5  1. ]
 [ 5.1  3.4  1.5  0.2  0. ]
 [ 5.8  2.7  4.1  1.   1. ]
 [ 4.9  2.5  4.5  1.7  2. ]
 [ 4.6  3.6  1.   0.2  0. ]
 [ 6.4  3.2  5.3  2.3  2. ]
 [ 4.4  2.9  1.4  0.2  0. ]
 [ 5.2  3.4  1.4  0.2  0. ]
 [ 5.1  3.8  1.6  0.2  0. ]]


### Scoring Function

Cart use GINI, C4.5 use Entropy

In [34]:
#Calculate the GINI index for split dataset
def gini_index(groups, classes):
    #count all samples at split point
    n = float(sum([len(group) for group in groups]))
    #sum weighted GINI for each group
    gini = 0.0
    for group in groups:
        size = float(len(group))
        if size == 0:
            continue
        score = 0.0
        #give the score to a group based on score of each class
        for class_x in classes:
            p = [row[-1] for row in group].count(class_x) / size
            score += p*p
            #weight the group score by its relative size
        gini += (1.0-score)*(size/n)
    return gini   

### Split data

In [40]:
def split_data(index, value, dataset):
    left, right = list(), list()
    for row in dataset:
        if row[index] < value:
            left.append(row)
        else: 
            right.append(row)
    return left, right

### Find the best split value using scoring function and split_data
Best split will reduce largest GINI index after spliting at that value, from parent node to children node, Each node of the tree will contain, the index of the features used for split, the value of the feature and the left/right groups

In [36]:
def get_split(dataset):
    class_values = list(set(row[-1] for row in dataset))
    b_index, b_value, b_score, b_groups = 999,999,999, None
    for index in range(len(dataset[0]) -1):
        for row in dataset:
            groups = split_data(index, row[index], dataset)
            gini = gini_index(groups, class_values)
            if gini < b_score:
                b_index, b_value, b_score, b_groups = index, row[index], gini, groups
    return {'index': b_index, 'value': b_value, 'groups': b_groups}

### leaf node
Get to the end of that branch of a tree(terminal node)

In [42]:
def to_leaf(group):
    outcomes = [row[-1] for row in group]
    return max(set(outcomes), key = outcomes.count)

### Recursively split

In [37]:
#Create children splits for a node or get to the end(hit the stop criteria)
def split(node, max_depth, min_size, depth):
    left, right = node['groups']
    del(node['groups'])
    #check for the end(no split)
    if not left or not right:
        node['left'] = node['right'] = to_leaf(left+right)
        return
    #check for max depth
    if depth >= max_depth:
        node['left'], node['right'] = to_leaf(left), to_leaf(right)
        return
    #for left node
    if len(left) <= min_size:
        node['left'] = to_leaf(left)
    else:
        node['left'] = get_split(left)
        split(node['left'], max_depth, min_size, depth + 1)
    #for right node
    if len(right) <= min_size:
        node['right'] = to_leaf(right)
    else:
        node['right'] = get_split(right)
        split(node['right'], max_depth, min_size, depth + 1)

### Build the tree

In [38]:
def build_tree(train, max_depth, min_size):
    root = get_split(train)
    split(root, max_depth, min_size, 1)
    return root

In [43]:
tree = build_tree(train_set, 2, 1)

### Print the tree

In [44]:
def print_tree(node, depth = 0):
    if isinstance(node, dict):
        print('%s[%s < %.3f]' % ((depth * "  ", (iris.feature_names[node['index']]),
                                  node['value'])))
        print_tree(node['left'], depth + 1)
        print_tree(node['right'], depth + 1)
    else:
        print('%s[%s]' % ((depth * "    ", node)))
        
        

In [45]:
print_tree(tree)

[petal length (cm) < 3.300]
  [sepal length (cm) < 4.700]
        [0.0]
        [0.0]
  [petal width (cm) < 1.700]
        [1.0]
        [2.0]


### Making prediction

In [46]:
def predict(node, row):
    if row[node['index']] < node['value']:
        if isinstance(node['left'], dict):
            return predict(node['left'], row)
        else:
            return node['left']
    else:
        if isinstance(node['right'], dict):
            return predict(node['right'], row)
        else:
            return node['right']   

### Test the preformance of decision tree

In [ ]:
y_pred = list()
for row in X_test:
    prediction = predict(tree, row)
    y_pred.appe